# Dataset & Dataloader
> 데이터세트는 데이터의 집합을 의미하며, 입력값(X)와 결과값(Y)에 대한 정보를 제공하거나 일련의 데이터 묶음을 제공한다.

## 데이터세트 클래스 기본형

- 초기화 메서드( __init__ ): 입력된 데이터의 전처리 과정을 수행

- 호출 메서드( __getitem__ ): 학습을 진행할 때 사용되는 하나의 행을 불러오는 과정

- 길이 반환 메서드( __len__ ): 학습에 사용된 전체 데이터세트의 개수를 반환한다.

In [1]:
class Dataset:
  def __init__(self, data, *arg, **kwargs):
    self.data = data
  
  def __getitem__(self, index):
    return tuple(data[index] for data in data.tensor)
  
  def __len__(self):
    return self.data[0].size(0)

## DataLoader
> Dataset에 저장된 데이터를 어떠한 방식으로 불러와 활용할지 정의한다.

### 제공
- 배치 크기(batch_size): 학습에 사용되는 데이터의 개수가 많이 한 번의 엑포에서 모든 데이터를 메모리에 올릴 수 없을 때 데이터를 나누는 역할을 수행한다.
- 데이터 순서 변경(shuffle): 모델이 데이터 간의 관계가 아닌, 데이터의 순서로 학습되는 것을 방지하고자 수행
- 데이터 로드 프로세스 수(num_workers): 데이터를 불러올 때 사용할 프로세스의 개수를 의미한다.

## 다중 선형 회귀

In [3]:
import torch
from torch import nn
from torch import optim
from torch.utils.data import TensorDataset, DataLoader

train_x = torch.FloatTensor([
  [1, 2], [2, 3], [3, 4], [4, 5], [5, 6], [6, 7]
])
train_y = torch.FloatTensor([
  [0.1, 1.5], [1, 2.8], [1.9, 4.1], [2.8, 5.4], [3.7, 6.7], [4.6, 8]
])

train_dataset = TensorDataset(train_x, train_y)
train_dataLoader = DataLoader(train_dataset, batch_size = 2, shuffle= True, drop_last= True)
# drop_last = True : 배치 크기에 맞지 않는 배치를 제거한다. 예를 들어, 데이터세트의 크기가 5일 때, 배치사이즈가 2라면 마지막 배치의 크기는 1이 된다. 1인 마지막 배치를 학습에 포함시키지 않는다.

# 모델, 오차함수, 최적화 선언
model = nn.Linear(2, 2, bias = True)
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr = 0.001)

# 데이터로더 적용
for epoch in range(20000):
  cost = 0.0
  
  for batch in train_dataLoader:
    x, y = batch
    output = model(x)
    
    loss = criterion(output, y)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    cost += loss
  
  cost = cost / len(train_dataLoader)
  
  if(epoch + 1) % 1000 == 0:
    print(f'Epoch {epoch + 1:4d} | Model {list(model.parameters())} | Cost: {cost:.3f}')


Epoch 1000 | Model [Parameter containing:
tensor([[0.5873, 0.1982],
        [0.6323, 0.5959]], requires_grad=True), Parameter containing:
tensor([-0.5168, -0.0938], requires_grad=True)] | Cost: 0.031
Epoch 2000 | Model [Parameter containing:
tensor([[0.7035, 0.1382],
        [0.7052, 0.5582]], requires_grad=True), Parameter containing:
tensor([-0.6929, -0.2043], requires_grad=True)] | Cost: 0.008
Epoch 3000 | Model [Parameter containing:
tensor([[0.7627, 0.1076],
        [0.7423, 0.5390]], requires_grad=True), Parameter containing:
tensor([-0.7827, -0.2606], requires_grad=True)] | Cost: 0.002
Epoch 4000 | Model [Parameter containing:
tensor([[0.7928, 0.0921],
        [0.7613, 0.5293]], requires_grad=True), Parameter containing:
tensor([-0.8284, -0.2893], requires_grad=True)] | Cost: 0.001
Epoch 5000 | Model [Parameter containing:
tensor([[0.8082, 0.0841],
        [0.7709, 0.5243]], requires_grad=True), Parameter containing:
tensor([-0.8517, -0.3039], requires_grad=True)] | Cost: 0.000


# 모델 / 데이터세트 분리

## 모듈 클래스
> 초기화 메서드와 순방향 메서드를 재정의하여 활용한다. 초기화 메서드는 신경망에 사용될 계층을 초기화하고, 순방향 메서드에서는 모델이 어떤 구조를 갖게 될지를 정의한다.

In [ ]:
# 모듈 클래스 기본형
class Model(nn.Module):
  def __init__(self):
    super().__init__() # 모듈 클래스의 속성을 초기화
    self.conv1 = nn.Conv2d(1, 20, 5)
    self.conv2 = nn.Conv2d(20, 20, 5)
  
  def forward(self, x):
    x = F.relu(self.conv1(x))
    x = F.relu(self.conv2(x))
    return x

In [8]:
# 비선형 회귀

import torch
import pandas as pd
from torch import nn
from torch import optim
from torch.utils.data import DataLoader, Dataset

class CustomDataset(Dataset):
  def __init__(self, file_path):
    df = pd.read_csv(file_path)
    self.x = df.iloc[:, 0].values
    self.y = df.iloc[:, 1].values
    self.length = len(df)
    
  def __getitem__(self, index):
    x = torch.FloatTensor([self.x[index] ** 2, self.x[index]])
    y = torch.FloatTensor([self.y[index]])
    return x, y
  
  def __len__(self):
    return self.length
    
class CustomModel(nn.Module):
  def __init__(self):
    super().__init__()
    self.layer = nn.Linear(2, 1)
    
  def forward(self, x):
    x = self.layer(x)
    return x
    
train_dataset = CustomDataset('../dataset/non_linear.csv')
train_dataloader = DataLoader(train_dataset, batch_size = 128, shuffle=True, drop_last=True)

device = "mps" if torch.backends.mps.is_available() else "cpu"
model = CustomModel().to(device)
criterion = nn.MSELoss().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.0001)

for epoch in range(10000):
  cost = 0.0
  
  for x, y in train_dataloader:
    x = x.to(device)
    y = y.to(device)
    
    output = model(x)
    loss = criterion(output, y)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    cost += loss
  cost = cost / len(train_dataloader)
  
  if(epoch+1) % 1000 == 0:
    print(f'epoch: {epoch + 1:4d}, model : {list(model.parameters())}, cost: {cost:.3f}')

    

epoch: 1000, model : [Parameter containing:
tensor([[ 3.1064, -1.7001]], device='mps:0', requires_grad=True), Parameter containing:
tensor([0.1082], device='mps:0', requires_grad=True)], cost: 0.142
epoch: 2000, model : [Parameter containing:
tensor([[ 3.1058, -1.7030]], device='mps:0', requires_grad=True), Parameter containing:
tensor([0.1411], device='mps:0', requires_grad=True)], cost: 0.136
epoch: 3000, model : [Parameter containing:
tensor([[ 3.1056, -1.7028]], device='mps:0', requires_grad=True), Parameter containing:
tensor([0.1711], device='mps:0', requires_grad=True)], cost: 0.118
epoch: 4000, model : [Parameter containing:
tensor([[ 3.1051, -1.7028]], device='mps:0', requires_grad=True), Parameter containing:
tensor([0.1986], device='mps:0', requires_grad=True)], cost: 0.108
epoch: 5000, model : [Parameter containing:
tensor([[ 3.1043, -1.7029]], device='mps:0', requires_grad=True), Parameter containing:
tensor([0.2237], device='mps:0', requires_grad=True)], cost: 0.102
epoch